**<div align="center"><span style="font-size:4em">Lösung</span></div>**
# Capacitated facility location

In diesem Notebook sollen Sie ein Ganzzahlprogramm implementieren, und zwar für das *capacitated facility location*-Problem. Wir erlauben dabei an jedem Standort, entweder ein kleines oder ein großes Depot zu eröffnen (aber nicht beides). Mehr zu dem Problem finden Sie in Abschnitt 5.7 des Skripts. Auch das Notebook 
[facility.ipnyb](https://colab.research.google.com/github/henningbruhn/opt1/blob/main/facility.ipynb)
wird Ihnen nützlich sein.

\begin{align}
	\min\quad & \sum_{i\in\mathcal F}y_i^kf_i^k+y_i^gf_i^g
+\sum_{i\in\mathcal F,j\in\mathcal D}x_{ij}c_{ij}, 
& y^{k},y^g\in\mathbb R^{\mathcal F},\, x\in\mathbb R^{\mathcal F\times\mathcal D}\\
	\textrm{unter}\quad & \sum_{j\in\mathcal D} x_{ij}\leq q^ky_i^k+q^gy_i^g 
& \textrm{für alle }i\in\mathcal F\\
& y_i^k+y_i^g\leq 1 & \textrm{für alle }i\in\mathcal F\\
	& \sum_{i\in\mathcal F}x_{ij}\geq 1 & \textrm{für alle }j\in\mathcal D\\
	& x_{ij}\in\{0,1\},\,y_i^k,y_i^g\in\{0,1\} & \textrm{für alle }i\in\mathcal F,\,j\in\mathcal D
\end{align}




Verwendet wird das <code>mip</code>-Paket. Dokumentation findet sich [hier.](https://python-mip.readthedocs.io/en/latest/)
Wenn Sie in Google Colab arbeiten, dann verwenden Sie die nächste Zelle so wie sie ist. Wenn Sie auf dem eigenen Rechner arbeiten und mip bereits installiert haben, dann löschen Sie die nächste Zelle oder kommentieren Sie per Raute den Installationsbefehl.

In [1]:
# für google colab
!pip install mip  # Löschen / Auskommentieren, wenn Sie mip bereits installiert haben.

Wir brauchen einige Pakete.

In [2]:
import mip  ## Bibliothek für lineare Programme und MIPs
import random
import math
import matplotlib.pyplot as plt
import time

## Erzeugung einer Zufallsinstanz

Wie immer brauchen wir eine Beispielsinstanz, die wir zufällig erzeugen. Wir platzieren Standorte und Kunden zufällig in der Ebene. Als Anschluss-/Betriebskosten nehmen wir dann den gewöhnlichen Abstand in der Ebene. Die Konstruktionskosten wählen wir zufällig. Die Instanz wird in einem [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) gespeichert. Das ist eine Datenstruktur, die per Schlüsselwort verschiedenste Daten aufnehmen kann. Insbesondere sind folgende Informationen in dem dictionary <code>inst</code> enthalten:
* <code>inst["facilities"]</code>: eine Liste mit den Ebenenkoordinaten der Standorte für Depots
* <code>inst["customers"]</code>: eine Liste mit den Ebenenkoordinaten der Kunden
* <code>inst["opening_costs_small"]</code>: eine Liste mit den Konstruktionskosten für kleine Depots
* <code>inst["opening_costs_large"]</code>: eine Liste mit den Konstruktionskosten für große Depots
* <code>inst["capacity_small"]</code>: die Kapazität eines kleinen Depots (gleich für alle kleinen Depots)
* <code>inst["capacity_large"]</code>: die Kapazität eines großen Depots (gleich für alle großen Depots)
* <code>inst["num_facilities"]</code>: Anzahl der Standorte
* <code>inst["num_customers"]</code>: Anzahl der Kunden

In [3]:
### Sie müssen diesen Code nicht verstehen. Wichtig ist nur, dass die Instanz in der Variablen inst gespeichert wird.
def rnd_instance(num_facilities=30,num_customers=1000):
    random.seed(42)
    inst={}
    inst["facilities"]=[(random.random(),random.random()) for _ in range(num_facilities)]
    inst["customers"]=[(random.random(),random.random()) for _ in range(num_customers)]
    inst["opening_costs_small"]=[random.randint(7,14) for _ in range(num_facilities)]  #[10]*num_facilities
    inst["opening_costs_large"]=[random.randint(20,34) for _ in range(num_facilities)]
    inst["capacity_small"]=math.floor(0.5*num_customers/num_facilities)
    inst["capacity_large"]=round(3*num_customers/num_facilities)
    inst["num_facilities"]=num_facilities
    inst["num_customers"]=num_customers
    return inst

def distance(facility,customer):
    px,py=facility
    qx,qy=customer
    return math.sqrt((px-qx)**2+(py-qy)**2)

# Die eigentliche Instanz:
inst=rnd_instance()

Wie kommen wir nun an die Instanzdaten ran? So:

In [4]:
inst["capacity_small"]

16

### Aufgabe: facilities
Implementieren Sie das obige Ganzzahlprogramm. Nutzen Sie den Code unten, um die Kosten und Eigenschaften der Lösung sowie die Rechenzeit auszugeben. Dazu legen Sie bitte im Ganzzahlprogramm folgende Variablen an:
* <code>opening_costs</code> für die Konstruktionskosten 
* <code>connection_costs</code> für die Betriebskosten 
* <code>y_k</code> eine Binärvariable pro Standort, um anzuzeigen, ob dort ein kleines Depot eröffnet wird
* <code>y_g</code> eine Binärvariable pro Standort, um anzuzeigen, ob dort ein großes Depot eröffnet wird

Am einfachsten ist die Aufgabe, wenn Sie den Code aus dem Notebook
[facility.ipnyb](https://colab.research.google.com/github/henningbruhn/opt1/blob/main/facility.ipynb)
adaptieren.

### Lösung



In [5]:
model=mip.Model()
num_customers=inst["num_customers"]
num_facilities=inst["num_facilities"]
facilities=inst["facilities"]
customers=inst["customers"]
y_g=[model.add_var(var_type=mip.BINARY) for _ in range(num_facilities)]
y_k=[model.add_var(var_type=mip.BINARY) for _ in range(num_facilities)]
x=[[model.add_var(var_type=mip.BINARY) for _ in range(num_customers)] for _ in range(num_facilities)]

opening_costs=mip.xsum(y_k[i]*inst["opening_costs_small"][i]+y_g[i]*inst["opening_costs_large"][i]  for i in range(num_facilities))
connection_costs=mip.xsum(x[i][j]*distance(facilities[i],customers[j]) for i in range(num_facilities) for j in range(num_customers))

model.objective=mip.minimize(opening_costs+connection_costs)

for i in range(num_facilities):
    for j in range(num_customers):
        model+=x[i][j]<=inst["capacity_small"]*y_k[i]+inst["capacity_large"]*y_g[i]
        
for i in range(num_facilities):
    model+= y_k[i]+y_g[i] <= 1
    
for j in range(num_customers):
    model+=mip.xsum(x[i][j] for i in range(num_facilities)) >= 1

Wir starten die Optimierung und messen die Rechenzeit.

In [6]:
start=time.time() # Code zur Zeitmessung
model.optimize()
end=time.time()
total_time=end-start

total_costs=model.objective_value
num_small_facilities=sum([y_k[i].x for i in range(num_facilities)])
num_large_facilities=sum([y_g[i].x for i in range(num_facilities)])

print("Gesamtkosten: {}".format(round(total_costs,1)))
print("  davon Betriebskosten: {}".format(round(connection_costs.x,1)))
print("  davon Konstruktionskosten: {}".format(round(opening_costs.x,1)))
print("Zahl großer / kleiner Depots: {} / {}".format(num_large_facilities,num_small_facilities))
print("Rechenzeit für Optimierung: {}s".format(round(total_time,1)))

Gesamtkosten: 228.6
  davon Betriebskosten: 165.6
  davon Konstruktionskosten: 63.0
Zahl großer / kleiner Depots: 0.0 / 7.0
Rechenzeit für Optimierung: 12.5s


### Aufgabe: Relaxierung
Implementieren Sie das Ganzzahlprogramm noch einmal, nur dieses Mal mit reellwertigen Variablen. Dh, ersetzen Sie jede Binärvariable durch eine, die die Werte $[0,1]$ annehmen kann. Damit erhalten Sie ein gewöhnliches lineares Programm, oft (lineare) *Relaxierung* des ursprünglichen Programms genannt. Lassen Sie die Optimierung laufen und vergleichen Sie die optimalen Kosten und die Rechenzeit mit dem Ganzzahlprogramm oben. Was beobachten Sie? Anworten Sie **kurz** in einer *markdown*-Zelle. (Für robuste Aussagen über die Rechenzeit müsste man Ganzzahlprogramm und Relaxierung beide wiederholt lösen lassen und über die Rechenzeiten mitteln. Das ersparen wir uns.)

### Lösung

In [7]:
model=mip.Model()
num_customers=inst["num_customers"]
num_facilities=inst["num_facilities"]
facilities=inst["facilities"]
customers=inst["customers"]
y_g=[model.add_var(ub=1) for _ in range(num_facilities)]
y_k=[model.add_var(ub=1) for _ in range(num_facilities)]
x=[[model.add_var(ub=1) for _ in range(num_customers)] for _ in range(num_facilities)]

opening_costs=mip.xsum(y_k[i]*inst["opening_costs_small"][i]+y_g[i]*inst["opening_costs_large"][i]  for i in range(num_facilities))
connection_costs=mip.xsum(x[i][j]*distance(facilities[i],customers[j]) for i in range(num_facilities) for j in range(num_customers))

model.objective=mip.minimize(opening_costs+connection_costs)

for i in range(num_facilities):
    for j in range(num_customers):
        model+=x[i][j]<=inst["capacity_small"]*y_k[i]+inst["capacity_large"]*y_g[i]
        
for i in range(num_facilities):
    model+= y_k[i]+y_g[i] <= 1
    
for j in range(num_customers):
    model+=mip.xsum(x[i][j] for i in range(num_facilities)) >= 1

Identischer Code wie oben.

In [8]:
start=time.time()
model.optimize()
end=time.time()
total_time=end-start

total_costs=model.objective_value
num_small_facilities=sum([y_k[i].x for i in range(num_facilities)])
num_large_facilities=sum([y_g[i].x for i in range(num_facilities)])

print("Gesamtkosten: {}".format(round(total_costs,1)))
print("  davon Betriebskosten: {}".format(round(connection_costs.x,1)))
print("  davon Konstruktionskosten: {}".format(round(opening_costs.x,1)))
print("Zahl großer / kleiner Depots: {} / {}".format(num_large_facilities,num_small_facilities))
print("Rechenzeit für Optimierung: {}s".format(round(total_time,1)))

Gesamtkosten: 109.5
  davon Betriebskosten: 102.5
  davon Konstruktionskosten: 7.0
Zahl großer / kleiner Depots: 0.25000000000000017 / 0.0
Rechenzeit für Optimierung: 0.1s


Was beobachten wir? Die Relaxierung lässt sich viel schneller lösen. Die relaxierte Lösung hat jedoch deutlich geringere Kosten und scheint nicht viel mit der Lösung des Ganzzahlprogramms gemein zu haben. (Man beachte die Zahl der konstruierten Depots.)